#### Try to move/migrate MySQL database created during the sessions to SQLite3

In [11]:
import sqlite3

def readSqliteTable():
    try:
        sqliteConnection = sqlite3.connect('example.db')
        cursor = sqliteConnection.cursor()
        print("Connected to SQLite")

        sqlite_select_query = """SELECT * from stocks"""
        cursor.execute(sqlite_select_query)
        records = cursor.fetchall()
        print("Total rows are:  ", len(records))
        print("Printing each row")
        for row in records:
            print("date: ", row[0])
            print("trans: ", row[1])
            print("symbol: ", row[2])
            print("qty: ", row[3])
            print("price: ", row[4])
            print("\n")

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to read data from sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()
            print("The SQLite connection is closed")

readSqliteTable()

Connected to SQLite
Total rows are:   1
Printing each row
date:  2006-01-05
trans:  BUY
symbol:  RHAT
qty:  100.0
price:  35.14


The SQLite connection is closed


#### Using MySQL or SQLite3 database : create a database which contains 2 tables: Countries and Cities (add the fields with the proper information about them - name,area, population, etc.). Each city should be linked to the country.

In [12]:
import mysql.connector

cnx = mysql.connector.connect(user='root',
                              password='',
                              host='localhost',
                              database='geo')

cursor = cnx.cursor()

query = "select name, area, population, country, code from geo.cities ci inner join geo.countries co on ci.id = co.id"
cursor.execute(query)

for row in cursor.fetchall():
    print(row)

cursor.close()
cnx.close()

('Kaunas', 157, 298753, 'Lithuania', 'LT')
('Vilnius', 401, 580020, 'Lithuania', 'LT')
('Riga', 307, 605802, 'Latvia', 'LV')
('Tallinn', 159, 460642, 'Estonia', 'EE')


#### Create the Python program which allows to Insert/Update from the local files and Export to the local files.

In [13]:
import pandas as pd
import pymysql

from sqlalchemy import create_engine
from sqlalchemy import text

In [14]:
update_countries = pd.read_csv(r"countries.csv")
update_countries.head()

,id,country,code
0,1,Lithuania,LT
1,2,Latvia,LV
2,3,Estonia,EE
3,4,Poland,PL


In [15]:
update_cities = pd.read_csv(r"cities.csv")
update_cities.head()

,city_id,name,area,population,id
0,1,Vilnius,401,580020,1
1,2,Riga,307,605802,2
2,3,Tallinn,159,460642,3
3,4,Kaunas,157,298753,1


In [16]:
engine = create_engine("mysql+pymysql://root@localhost/geo")
update_countries.to_sql(name="countries",con=engine,if_exists="replace",index=False)

4

In [18]:
update_cities.to_sql(name="cities",con=engine,if_exists="replace",index=False)

4

In [19]:
#Export to the local files

engine = create_engine("mysql+pymysql://root@localhost/geo")


with engine.begin() as con:
    query = text("SELECT * FROM cities")
    df_cities = pd.read_sql(query,con)
    df_cities.to_csv('cities.csv', index=False)

In [20]:
with engine.begin() as con:
    query = text("SELECT * FROM countries")
    df_countries = pd.read_sql(query,con)
    df_countries.to_csv('countries.csv', index=False)